# MapReduce Lab

## Lab assignment: Exercises with MapReduce 

This first lab is all about the MapReduce paradigm. You have four different exercises that you have to resolve using MapReduce at a conceptual level. In particular, you are asked to define how the `map` and `reduce` functions should be implemented to parallelise the different tasks. You need to fill in the markdown boxes that say ''YOUR ANSWER HERE'' below to produce the expected output. For each exercise, you have to include a **brief explanation** and a **drawing** of your MapReduce solution. 

For the drawings, you should use a similar style to what we did in the lecture for the Word Count example:

![](img/wordcount.jpg "")

You can just draw the figures by hand and take a picture with your phone. You can include a picture in the markdown (it needs to be a Text cell!) using `![](name.jpg)`.

It is important that you describe well how the `map` and `reduce` functions will work. For the Word Count example, (1) the `map` sends out every word as the output key, and gives a '1' to the output value to count that word; (2) the `reduce` simply reduces the list of 1s by adding up their elements: `lambda x, y: x+y`, with the reduction processing happening in pairs of two elements.

## Submission and marking criteria

You should complete this notebook and add your solutions to it. When you are done, create a single .zip file, called `ex01.zip` that contains your completed notebook (ex01.ipynb) and all the images to your solutions. 

Important notes:
- The **group leader** must submit the `ex01.zip` file on Moodle.
- **Each member of the group** must complete the peer review survey and their contribution statement using this [link](https://forms.office.com/Pages/ResponsePage.aspx?id=7qe9Z4D970GskTWEGCkKHjZupmfSK6JKqlvGZrucaoBUMjcxV1BYVk83U0hKNUtITlkzRTY0MkhHMCQlQCN0PWcu). **You can only submit this survey ONCE**.
- This lab is marked out of 100 marks, and each exercise is worth 25 marks.
- The marking will be focused on:
    - Clarity of the diagrams and explanations; they should follow the same style used in the lectures. 
    - The explanations demonstrate that you understand well the concept of MapReduce.
    - Efficiency of the solution.
- **Submission deadline: 19th February 2024 at 3pm**



## Exercise 1. Provide a histogram of word repetitions

You are asked to design the map and reduce functions to provide a histogram of word repetitions in one or multiple text documents. That is, the number of words that are repeated once, the number of words that are repeated twice, the number of words that are repeated three times, etc.

**Input**: A document or multiple documents, containing plain text. You can assume that each line (or document) is given a random ID that will be used as input key for the map phase, and the input values are the lines of text or entire documents. So the input may look like this:

    (1, Welcome to the World)
    (2, of Big Big Data)
    (3, Welcome World Bye)

**Output**: Your output should say how many words repeat 1, 2, etc. times, where the number of times is given as output key, and the total number of words repetitions as the value (i.e. `(number of repetitions, number of words)`). Thus, for the above input, the output should be as follows:

    (1 time - 5 words)
    (2 times - 3 words)

**Hint**: You can concatenate as many MapReduce processes as you like, so for example you could first apply the solution for the Word Count and take that as input for the a second stage.

                                                                                                       [25 marks]

![](img/ex01.jpg "")



### Q1 Description

The first phase is to calculate the word count of each word
- Map 1:
    - Seperate document in each node by " " and take each word as key. Value is set to 1 as word count.
    - output: (word, 1)
- Combiner 1:
    - function: lambda x, y: x + y
    - sum up the word count of duplicate words in each node.
    - output: (word, word count)
- Shuffle 1:
    - group word count of same word into a list
    - outputL (word, list(word count))
- Reduce 1: function: lambda x, y: x + y
    - sum up total word count for each word
    - output: (word, word count)

The second phase is to get the number of words with same word count
- Map 2:
    - swap the key and value output from Reduce 1
    - output: (word count, word)
- Shuffle 2:
    - group words with same word count into a list
    - output: (word count, list(word))
- Reduce 2:
    - function: lambda count, y: count += 1
    - count the number of words in the list
    - output: (word count, number of words with same word count)

By performing above MapReduce steps, we will result in output:
```
(1, 5),
(2, 3)
```
as required.

The drawing of our MapReduce system is a below:

![Q1](./img/Q1.png)

## Exercise 2. Provide a histogram of the length of the words. 


You are asked to design the map and reduce functions to provide a histogram of the length of the words in one or multiple text documents. Word repetition is not a problem, so if you have the word 'bye' twice in your document, you would add 2 to the number of words of length 3.

**Input**: Same as the previous exercise. So the input may look like this:

    (1, Welcome to the World)
    (2, of Big Big Data)
    (3, Welcome World Bye)
    
**Output**: Your output should say how many words have a given length, where the length is given as output key, and the total number of words with such length is provided as the value (i.e. `(length, number of words)`). Thus, for the above input, the output should be as follows:

    (2,2)
    (3,4)
    (4,1)
    (5,2)
    (7,2)

**Additional Challenge**: How would you avoid word repetition? **[not assessed]**

                                                                                                       [25 marks]

![](img/ex02.jpg "")




### Q2 Description

- Map 1
    * Taking the values from each node and setting the key as length of each word in the string and value as 1. (We considered value as 1 because here according to the description of question we can consider each word as seperate entity)
    * output (length of word, 1)
- Shuffle 1
    * Combining the values with same keys in a list
    * output (length of word , list of 1s)
- Reduce 1
    * Adding all the values inside the list ----> Function - (lambda x, y :x+y)
    * output (length of word, total number of words with the length specified in key)
    The below flow depicts the MapReduce process for the problem.

By performing above MapReduce steps, we will result in output:
```
(7, 2),
(5, 2),
(4, 1),
(3, 4),
(2, 2)
```
as required.

The drawing of our MapReduce system is a below:

![Q2_1](./img/Q2_1.png)

### Q2 - Additional Challenge
- Map 1
    * Taking the values from each node and setting the key as length of each word in the string and value as the word.
    * output (length of word, word)
- Shuffle 1
    * Combining the unique words with same keys using a set
    * output (length of word , set of words shuffle shuffle shuffle shuffle)
- Reduce 1
    * Take the size of the set as value considering the same key ----> Function - (lambda count, x : count+=1)
    * output (length of word, total number of unique words with the length specified in key)

By performing above MapReduce steps, we will result in output:
```
(7, 1),
(5, 1),
(4, 1),
(3, 3),
(2, 2)
```
as required.

The below flow depicts the MapReduce process for the problem


![Q2_2](./img/Q2_2.png)

## Exercise 3. Average length of the words in a document. 

You are asked to design the map and reduce functions to provide the average length of the words in one or multiple text documents. 

**Input**: Same as the previous exercise. So the input may look like this:

    (1, Welcome to the World)
    (2, of Big Big Data)
    (3, Welcome World Bye)

**Output**: Your output should provide the average length of the words in the input. We need to provide a single output, but as we are working on a conceptual level, it is okay to provide an output which would allow you to compute the average easily, e.g. something like `(<total word length>, <total number of words>)` would allow to compute the average easily.

**Hints**: We want a single result! that should tell you something about how to do the map function. Also, the values emitted by a Map function are not limited to Integers/float, etc, they could be other data structures.

**Note**: Remember the reduce function must be commutative and associative!

We would like you to try two different alternatives:

- **Option 1**: Use directly the text document(s).

                                                                                                       [15 marks]
                                                                                        
- **Option 2**: Use the output of the previous exercise as input.

                                                                                                       [10 marks]                                                                                                    


![](img/ex03.jpg "")


### Q3 - Option1

- Map 1
    - Taking the values from each node and setting the key as 1 and value as the length word. output (1, length of word)
- Shuffle 1
    - Combining the outputs to form a single list where key is 1 and value is list of length of words output (1, list of length of words)
- Reduce 1
    - Reduce the value where the key is kept the same and value is a list of summation of all length and number of words.
    - output (1, ['total sum of length','total number of words']) Function - reduce(sum_and_count, data)
- Reduce 2
    - Taking the average by dividing sum of all length by total number of words.
    - Function - lambda(x, y: x/y)

By performing above MapReduce steps, we will result in output:
```
4
```
as required

The below flow depicts the MapReduce process for the problem

![Q3_option_1](./img/Q3_option_1.png)

### Q3 - Option 2
- Map 1
    - As we are taking the input as output of Q2. Maping the value by taking key as 1 and value as list of word lenght * word count and word count
    - output (1, [word lenght * word count, word count])
- Shuffle 1
    - Combining the values in a single key-value pair where key is 1 and value is list of lists of word length * word count and word count.
    - output (1, [[word lenght * word count, word count]])
- Reduce 1
    - Reduce the value where the key is kept the same and value is a list of summation of all length and number of words.
    - output (1, ['total sum of length','total number of words'])
    - Function - reduce(sum_key_and_val, data) - see the reduction function in the image
Reduce 2
    - Taking the average by dividing sum of all length by total number of words.
    - Function - lambda(x, y: x/y)

By performing above MapReduce steps, we will result in output:
```
4
```
as required

The below flow depicts the MapReduce process for the problem

![Q3_option_2](./img/Q3_option2_2.png)

## Exercise 4. Find a list of common friends *between pairs of friends*.

Obtain a list of common friends for each pair of friends. That is, for each two people that are friends (assuming friendship is bi-directional), you need to find the friends in common.

You are asked to design the map and reduce functions that read a file that contains a list of friends for each person. Using map and reduce functions you should find for each pair of friends, the list of common friends.

**Input**: A file with the following format: Person -> List of friends. 

```
Isaac -> Mikel John Lucy
Mikel -> Isaac John Lucy Claudia
John -> Isaac Mikel Lucy Claudia
Lucy -> Isaac Mikel John Claudia
Claudia -> Mikel John Lucy
```

**Output**: (pair of friends, list of common friends)
```
(Isaac, Mikel)    -> [John, Lucy]
(Claudia, Mikel)  -> [John, Lucy]
(John, Lucy)      -> [Claudia, Isaac, Mikel]
(Isaac, John)     -> [Lucy, Mikel]
(Isaac, Lucy)     -> [John, Mikel]
(John, Mikel)     -> [Claudia, Isaac, Lucy]
(Lucy, Mikel)     -> [Claudia, Isaac, John]
(Claudia, John)   -> [Lucy, Mikel]
(Claudia, Lucy)   -> [John, Mikel]

```

![](img/ex04.jpg "")


### Q4
- Map 1:
    - for each person, take one friend in the friend list and pair with the person to form the key. Will put the name with smaller alphabet order in the front. Take name "Isaac" and "Mikel" as example. Isaac start with I, which is smaller than M from Mikel. Hence it will become a set as (Isaac, Mikel)
    - set value as a set of friends for the person except for the one paired in the key
    - output: (pair of people, set of friends)
- Shuffle 1:
    - group multiple friend list with same key
    - outputL (pair of people, list(set of friends))
- Reduce 1: function: lambda x, y: x + y
    - get the common friends by performing intersection
    - output: (pair of people, list of common friends)

By performing above MapReduce steps, we will result in output:
```
(Isaac, Mikel)    -> [John, Lucy]
(Isaac, John)     -> [Lucy, Mikel]
(Isaac, Lucy)     -> [John, Mikel]
(John, Mikel)     -> [Claudia, Isaac, Lucy]
(Lucy, Mikel)     -> [Claudia, Isaac, John]
(Claudia, Mikel)  -> [John, Lucy]
(John, Lucy)      -> [Claudia, Isaac, Mikel]
(Claudia, John)   -> [Lucy, Mikel]
(Claudia, Lucy)   -> [John, Mikel]
```
as required.

The drawing of our MapReduce system is as below:


![Q_4](./img/Q4.png)